# YOLOX 依存パッケージインストール(YOLOX Dependent Package Install)

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX

In [ ]:
%cd YOLOX

!pip install -U pip && pip install -r requirements.txt
!pip install -v -e .  

# NVIDIA APEXインストール(NVIDIA APEX Install) 

In [ ]:
# ToDo：将来的にColaboratoryのCUDA対応バージョンが変わった際にはインストールバージョンを変える必要あり
# ToDo：If the CUDA compatible version of Colaboratory changes in the future, it is necessary to change the installation version
!pip uninstall -y torch torchvision torchaudio
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%cd /content/
!git clone https://github.com/NVIDIA/apex

%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

# PyCocoToolsインストール(PyCocoTools Install)

In [ ]:
!pip install cython
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# データセットダウンロード(Download Dataset)

In [ ]:
%cd /content/

use_sample_image = True

if use_sample_image:
    !git clone https://github.com/Kazuhito00/YOLOX-Colaboratory-Training-Sample.git

# 学習/検証データ分割(Train/Validation split data)

In [ ]:
import os

# 独自のデータを使用する場合は、パスを指定してください
# Please fill in the path if you want to use your own data
if use_sample_image:
    dataset_directory = 'YOLOX-Colaboratory-Training-Sample/02.annotation_data'
else:
    dataset_directory = ''

# 学習/検証データパス(train/validation data path)
train_directory = './train'
validation_directory = './validation'

# 学習データ格納ディレクトリ作成(Create training data storage directory)
os.makedirs(train_directory, exist_ok=True)
# 検証データ格納ディレクトリ作成(Create verification data storage directory)
os.makedirs(validation_directory, exist_ok=True)

In [ ]:
import glob
import shutil
import random

# 学習データの割合(Percentage of training data)
train_ratio = 0.8

# コピー元ファイルリスト取得(Get copy source file list)
annotation_list = sorted(glob.glob(dataset_directory + '/*.xml'))
image_list = sorted(glob.glob(dataset_directory + '/*.jpg'))

file_num = len(annotation_list)

# インデックスシャッフル(shuffle)
index_list = list(range(file_num - 1))
random.shuffle(index_list)

for count, index in enumerate(index_list):
    if count < int(file_num * train_ratio):
        # 学習用データ(Training Data)
        shutil.copy2(annotation_list[index], train_directory)
        shutil.copy2(image_list[index], train_directory)
    else:
        # 検証用データ(Validation Data)
        shutil.copy2(annotation_list[index], validation_directory)
        shutil.copy2(image_list[index], validation_directory)

# Pascal VOC形式 を MS COCO形式へ変換(Convert Pascal VOC format to MS COCO format)

In [ ]:
!git clone https://github.com/Kazuhito00/voc2coco.git

In [ ]:
!python voc2coco/voc2coco.py train train/train_annotations.json
!python voc2coco/voc2coco.py validation validation/validation_annotations.json

# 学習データディレクトリ準備(Training data directory preparation)

In [ ]:
!mkdir dataset
!mkdir dataset/images
!mkdir dataset/images/train2017
!mkdir dataset/images/val2017
!mkdir dataset/annotations

!cp -rf train/*.jpg dataset/images/train2017
!cp -rf validation/*.jpg dataset/images/val2017
!cp -rf train/train_annotations.json dataset/annotations
!cp -rf validation/validation_annotations.json dataset/annotations

# コンフィグコピー
<!--
![image](https://user-images.githubusercontent.com/37477845/135283504-254ea817-345e-4665-828a-4c6034645ed1.png)
-->


In [ ]:
if use_sample_image:
    !cp /content/YOLOX-Colaboratory-Training-Sample/03.config/nano.py /content/YOLOX

# モデル訓練

In [ ]:
%cd /content/YOLOX/
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pth

In [ ]:
!cp tools/train.py ./

In [ ]:
!python train.py \
    -f nano.py \
    -d 1 \
    -b 16 \
    --fp16 \
    -o \
    -c yolox_nano.pth

# 推論テスト(Inference test)

In [ ]:
TEST_IMAGE_PATH = "/content/YOLOX-Colaboratory-Training-Sample/01.image/000050.jpg"
MODEL_PATH = "/content/YOLOX/YOLOX_outputs/nano/best_ckpt.pth"

!python tools/demo.py image \
    -f nano.py \
    -c {MODEL_PATH} \
    --path {TEST_IMAGE_PATH} \
    --conf 0.25 \
    --nms 0.45 \
    --tsize 640 \
    --save_result \
    --device gpu

In [ ]:
# from PIL import Image

# OUTPUT_IMAGE_PATH = "/content/YOLOX/YOLOX_outputs/nano/vis_res/2021_09_29_17_46_56/000050.jpg" 
# Image.open(OUTPUT_IMAGE_PATH)

# ONNX出力(Export ONNX Model)

In [ ]:
!python tools/export_onnx.py \
    --output-name yolox_nano.onnx \
    -n yolox-nano \
    -c {MODEL_PATH}

In [ ]:
!python demo/ONNXRuntime/onnx_inference.py \
    -m yolox_nano.onnx \
    -i {TEST_IMAGE_PATH} \
    -o ./ \
    -s 0.3 \
    --input_shape 416,416

In [ ]:
from PIL import Image

OUTPUT_IMAGE_PATH = "000050.jpg" 
Image.open(OUTPUT_IMAGE_PATH)